In [1]:
import pandas as pd
from src.utils import *
from src.pachage_list import *

In [2]:
df = pd.read_csv("../input/train.csv")

In [3]:
df

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate
...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective


In [5]:
df['essay_text'] = df['essay_id'].apply(lambda x: get_essay(x, is_train=True))
df['discourse_text'] = df['discourse_text'].apply(lambda x: resolve_encodings_and_normalize(x))
df['essay_text'] = df['essay_text'].apply(lambda x: resolve_encodings_and_normalize(x))
df['target'] = df['discourse_effectiveness'].map(LABEL_MAPPING)
df['text'] = df['discourse_type'] + ' ' + df['discourse_text'] + " " + df['essay_text']

In [6]:
df

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,essay_text,target,text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,"Hi, i'm Isaac, i'm going to be writing about h...",1,"Lead Hi, i'm Isaac, i'm going to be writing ab..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,"Hi, i'm Isaac, i'm going to be writing about h...",1,"Position On my perspective, I think that the f..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h...",1,Claim I think that the face is a natural landf...
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,"Hi, i'm Isaac, i'm going to be writing about h...",1,"Evidence If life was on Mars, we would know by..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h...",1,Counterclaim People thought that the face was ...
...,...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,Some people may ask multiple people for advice...,1,Claim For many people they don't like only ask...
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,Some people may ask multiple people for advice...,1,Claim also people have different views and opi...
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,Some people may ask multiple people for advice...,1,Position Advice is something that can impact a...
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,Some people may ask multiple people for advice...,0,Evidence someone can use everything that many ...


In [7]:
max_len = 0
length = []
lengths_dict = {}
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-large')
for text_col in ['discourse_text', 'essay_text']:
    lengths = []
    tk0 = tqdm(df[text_col].fillna("").values, total=len(df))
    for text in tk0:
        length = len(tokenizer(text=text, add_special_tokens=False)['input_ids'])
        lengths.append(length)
    lengths_dict[text_col] = lengths

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 36765/36765 [01:17<00:00, 476.38it/s]


In [9]:
max(lengths_dict['discourse_text']) + max(lengths_dict['essay_text']) + 2

2512

In [10]:
max(lengths_dict['discourse_text'])

907

In [11]:
max(lengths_dict['essay_text'])

1603